# Transformers language models



## Where do transformers live?

Open source TLMs are hosted in the [Hugginface Hub](https://huggingface.co/).

We can access them via their identification code through the hugginface API.

To do that it is necessary to install the transformers python library

In [ ]:
! pip install transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline, set_seed
set_seed(42)
import warnings
warnings.filterwarnings('ignore')

# (Extremely) brief overview of a Transformer block



Transformers are made of several identical blocks. They are not a unique NN but a stacking of blocks (layers) which can be further subdivided in sub-block (sub-layers)

A single building block is composed of two key elements:
- An attention layer. Where attention heads make their computations.
- A feed forward layer. A deep neural network.

![img](https://jalammar.github.io/images/t/Transformer_encoder.png)

*Picture taken from https://jalammar.github.io/illustrated-transformer/*

# BERT



![img](https://sesameworkshop.org/wp-content/uploads/2023/03/presskit_ss_bio_bert-560x420.png)
*picture from Google*

BERT is a TLM based on encoders.

- It is trained on the *Masked Language Modelling* and *sentence prediction* tasks.

- It can deal with different contexts, positional information and OOV.

BERT has many different variants that improves or modify the standard one or are for specific languages. (i.e. Distil-BERT, RoBERTa, UmBERTo etc..).


We will use a version called *Distil-BERT* which is a "distilled" version of the original, making the latter smaller withouth loosing much in performances.

## BERT inputs workflow

The workflow to get embeddings in BERT is likewise:
- *Tokenize* raw texts
- Add two special tokens: [CLS] at the beginning and [SEP] at the end
- *encode* tokens with a *word id* (pre-implemented). Word ids are already present in the vocabulary of the model for known words. For unknown words a special token [UNK] will be assigned (if they cannot be decomposed).
- run the model on the prepared *encodings*


![img](https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png)


*picture from [Jay Alammar awesome blog](https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)*




We can do that easily with *AutoModel* and *AutoTokenizer* classes

The library takes care of segmenting the text into tokens, converting it to ids and feed the model with the token embedding and the positional embedding (which both result from the pre-training)

In [ ]:
#instantiate the model and the tokenizer
model_id = "distilbert/distilbert-base-cased" #find all the ids on the hugginface hub
model = AutoModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Note on special tokens:
- [CLS] token at the beginning. This is also known as the classifier token and it is a *global representation* of the whole sentence. We can use it to build classification systems over sentences and short text.
- [SEP] token at the end of the sentences.

Both this tokens are used during pretraining for the Next Sentence Prediction task.




The code take car of doing it automatically

In [ ]:
#instantiate and encode some text
text = "I went to the gym yesterday morning. I was surrounded by people"
encoding_tokens = tokenizer(text, return_tensors = "pt")

### Brief note on tokenization

Tokenization in BERT models happens at the sub-lexical level. Thus, when a word is unknown, the tokenizer tries to break it down into pieces in the first place

In [ ]:
for n, id in enumerate(encoding_tokens["input_ids"][0]):
  print(n,tokenizer.decode(id),"---->", id.item())

0 [CLS] ----> 101
1 I ----> 146
2 went ----> 1355
3 to ----> 1106
4 the ----> 1103
5 gym ----> 10759
6 yesterday ----> 8128
7 morning ----> 2106
8 . ----> 119
9 I ----> 146
10 was ----> 1108
11 surrounded ----> 4405
12 by ----> 1118
13 people ----> 1234
14 [SEP] ----> 102


In [ ]:
text2 = "Ciao mi chiamo Mattia e studio linguistica computazionale"
enc = tokenizer.encode(text2)
for tok in enc:
  print(tokenizer.decode(tok))

[CLS]
C
##ia
##o
mi
ch
##iam
##o
Matt
##ia
e
studio
linguistic
##a
com
##put
##azi
##onal
##e
[SEP]


In [ ]:
#special token for unknown words
print("Our special token for unknown words is: ")
print(tokenizer.encode("[UNK]", add_special_tokens = False))

# subword tokenization
print("Tokenization example at the sub-word level: ")
print(tokenizer.tokenize("rumination"))


Our special token for unknown words is: 
[100]
Tokenization example at the sub-word level: 
['r', '##umi', '##nation']


We can inspect the vocabulary of the model, check for its length and have a look at words at random position

In [ ]:
print(f"The vocabulary of {model_id} has {len(tokenizer.vocab.keys())} tokens\n")

print("Words-->Word id\n")
for k in list(tokenizer.vocab.keys())[500:530]:
  print(k, "--->", tokenizer.vocab[k])


The vocabulary of distilbert/distilbert-base-cased has 28996 tokens

Words-->Word id

whether ---> 2480
Snow ---> 8442
́ ---> 389
##rkin ---> 17687
squares ---> 16004
fierce ---> 9250
concurrently ---> 18061
##ey ---> 2254
よ ---> 925
##ishing ---> 10506
Wednesday ---> 9031
##tee ---> 26032
circumstances ---> 5607
spends ---> 16994
Victoria ---> 3006
wonderful ---> 7310
ʔ ---> 375
million ---> 1550
264 ---> 23852
##oli ---> 11014
endorsed ---> 11889
##nko ---> 17075
umpire ---> 25077
##ronic ---> 26003
##sla ---> 26597
pump ---> 11188
faculties ---> 22094
silently ---> 8490
Own ---> 13432
Holding ---> 14382


## Extract the embeddings

In [ ]:
#run the model over our input
text = "I went to the gym yesterday morning. I was surrounded by people"
encoding_tokens = tokenizer(text, return_tensors = "pt")

with torch.inference_mode():
  outputs = model(**encoding_tokens)

In [ ]:
# access the shape of our output
shape = outputs["last_hidden_state"][0].shape
print(f"The shape of our output is: {shape}\n")
print(f"This means we have {shape[0]} words represented with vectors of dimension {shape[1]}")

The shape of our output is: torch.Size([15, 768])

This means we have 15 words represented with vectors of dimension 768


We have 15 words because we added 2 special tokens to our initial 13

In [ ]:
# extract the embeddings and show them in association with the corresponding word
embedded_words = []
word_embeddings = outputs["last_hidden_state"][0]
words = [tokenizer.decode(i) for i in encoding_tokens["input_ids"][0]]
embedding_df = pd.DataFrame(word_embeddings).astype(float)
embedding_df.index = words
embedding_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
[CLS],0.333347,0.043025,-0.079218,-0.084207,-0.212797,-0.123895,0.252003,-0.174414,-0.056993,-0.938937,...,0.418048,0.361712,-0.152836,-0.049004,-0.023088,0.267946,-0.060299,-0.304645,0.117908,0.014567
I,0.262142,-0.338454,0.300905,0.009044,-0.121172,0.125886,0.553342,-0.232121,0.055819,-0.433263,...,0.184417,0.123045,-0.365363,-0.058124,-0.058661,0.492310,0.640887,-0.170047,-0.000576,0.093725
went,0.342653,0.337142,0.351990,-0.095413,0.628846,-0.074906,0.209897,-0.025465,-0.541057,0.518051,...,0.103655,0.074183,-0.105410,-0.054341,-0.032809,0.474045,0.135485,-0.024260,-0.264745,0.180085
to,0.455950,0.422148,-0.056485,0.062724,0.584960,-0.182029,0.491096,0.093366,-0.322163,0.311484,...,0.223978,0.320306,-0.295646,-0.386567,0.180663,0.154933,-0.648991,-0.136615,-0.015364,-0.158476
the,-0.015327,-0.012505,-0.052235,0.256436,0.432003,-0.038705,0.504136,-0.287105,-0.577677,0.358360,...,0.339154,0.339906,-0.247194,-0.203175,0.147722,0.481793,-0.278707,0.074257,-0.192144,0.184582
gym,0.089551,0.121530,0.052608,0.033602,0.075624,-0.006596,-0.027430,0.027705,-0.276591,0.242132,...,-0.033289,0.086502,0.186209,0.027672,-0.500361,-0.005887,0.241406,-0.473748,-0.241873,0.122401
yesterday,-0.086992,-0.345998,0.316899,0.166045,0.384935,0.181641,0.293512,-0.504798,-0.316557,0.345682,...,0.001970,0.066779,-0.031057,-0.040903,-0.222445,-0.081098,-0.281266,-0.349598,-0.159604,0.277140
morning,0.164726,-0.213105,0.094032,0.102668,0.483439,-0.035999,0.196843,-0.146765,-0.121567,0.396554,...,-0.010726,0.392034,-0.214797,-0.081358,0.129133,-0.066733,-0.443217,0.017861,0.094103,0.289514
.,0.507484,0.203033,0.122107,0.714107,0.362723,-0.125391,0.446712,-0.157362,-0.207219,0.186249,...,-0.095349,0.029737,-0.156662,-0.464045,-0.173531,0.418284,0.008564,-0.067754,0.342219,-0.132604
I,-0.030914,-0.056213,-0.224483,0.306893,0.199719,-0.091678,0.268608,-0.222513,-0.034011,0.325369,...,0.099230,0.136378,-0.047534,0.040639,-0.135952,0.340671,0.544617,0.044454,0.008186,0.174646


Different contexts yileds different embeddings.

A simple visualization of the differences between three embeddings of the same word:

We take two sentences as:
- We went to the river *bank*
- I went to the *bank* to make a deposit

In [ ]:
@torch.inference_mode()
def get_token_repr(s,id):
  enc = tokenizer(s, return_tensors = "pt")
  position = [i.item() for i in enc["input_ids"][0]].index(id)
  print(position, enc["input_ids"][0][position])
  outputs = model(**enc)["last_hidden_state"]
  return outputs.squeeze(dim = 0)[position]

bank1 = "We went to the river bank."
bank2 = "I need to go to the bank to make a deposit."

#encode the word of our interest
word_id = tokenizer.encode("bank", add_special_tokens = False)[0]
#print(word_id)

#compare bank1 and bank 2 and bank no context
bank1_emb = get_token_repr(bank1, word_id)
bank2_emb = get_token_repr(bank2, word_id)
bank_non_context = model.embeddings.word_embeddings(torch.tensor(word_id)).detach()

#create a dataframe with the three tensors
bank_df = pd.DataFrame(torch.stack((bank_non_context, bank1_emb,bank2_emb))).astype(float)
bank_df.index = ["bank.no_cntxt","bank.1", "bank.2"]
bank_df.style.background_gradient(cmap = "Blues", vmin = -1, vmax = 1)



6 tensor(3085)
7 tensor(3085)


# The mechanism behind the magic: Attention

The *Self-Attention* mechanism allows these models to generate word representations that take into account the surrounding elements.

The picture below shows an example of it. You can see how the word "it" *attends* to other previous and successive words, all of which contribute to the dynamic representation of the current word.


![img](https://jalammar.github.io/images/t/transformer_self-attention_visualization.png)

*Picture from https://jalammar.github.io/illustrated-transformer/*

The mechanism itself is a complex matrix multiplication operation.
Each input token is represented with  three different matrices learned during the training process:
- Queries
- Keys
- Values

They are eventually multiplied together and normalized over the number of attention heads to get the inner representation of the word $t$ at layer $n$ before passing it to the Feed Forward Network

![img](https://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png)

*Picture from https://jalammar.github.io/illustrated-transformer/*


Try https://huggingface.co/spaces/exbert-project/exbert for more high level visualizations of attention in various models!

# Some Practical example using BERT

## Fill-Mask


<div>
  <img src="https://media.geeksforgeeks.org/wp-content/uploads/20200422002516/maskedLanguage.jpg" width="600">
</div>

*picture from https://media.geeksforgeeks.org/

We can replicate the mask language modeling objective using the 'pipeline' object of the transformers library

[*pipeline*](https://huggingface.co/docs/transformers/en/main_classes/pipelines) is an object that leverage the pre-trained model or a specific fine-tuned one and takes care of all the preparation steps. It is mostly used for inference.

In [ ]:
#instantiate the fill-mask pipeline
fill_mask = pipeline('fill-mask', model= model_id)

In [ ]:
#helper function to get the prediction
def get_fillers(s, output_all = False):
    if not output_all:
      print(fill_mask(s)[0]["sequence"])
    else:
      return fill_mask(s)

In [ ]:
fill_mask("Artificial Intelligence [MASK] take over the world.")

[{'score': 0.1016608253121376,
  'token': 1209,
  'token_str': 'will',
  'sequence': 'Artificial Intelligence will take over the world.'},
 {'score': 0.0328458696603775,
  'token': 2088,
  'token_str': 'forces',
  'sequence': 'Artificial Intelligence forces take over the world.'},
 {'score': 0.0315660759806633,
  'token': 1169,
  'token_str': 'can',
  'sequence': 'Artificial Intelligence can take over the world.'},
 {'score': 0.01982802338898182,
  'token': 5789,
  'token_str': 'agents',
  'sequence': 'Artificial Intelligence agents take over the world.'},
 {'score': 0.01851055398583412,
  'token': 1106,
  'token_str': 'to',
  'sequence': 'Artificial Intelligence to take over the world.'}]

Check for bias (still)...

In [ ]:
print(fill_mask("In the hospital the woman works as a [MASK]."))
print(fill_mask("In the hospital the man works as a [MASK]."))

[{'score': 0.7329415678977966, 'token': 7439, 'token_str': 'nurse', 'sequence': 'In the hospital the woman works as a nurse.'}, {'score': 0.028688404709100723, 'token': 3995, 'token_str': 'doctor', 'sequence': 'In the hospital the woman works as a doctor.'}, {'score': 0.028663545846939087, 'token': 13487, 'token_str': 'maid', 'sequence': 'In the hospital the woman works as a maid.'}, {'score': 0.028032952919602394, 'token': 23722, 'token_str': 'cleaner', 'sequence': 'In the hospital the woman works as a cleaner.'}, {'score': 0.025764331221580505, 'token': 26458, 'token_str': 'housekeeper', 'sequence': 'In the hospital the woman works as a housekeeper.'}]
[{'score': 0.19614218175411224, 'token': 7439, 'token_str': 'nurse', 'sequence': 'In the hospital the man works as a nurse.'}, {'score': 0.1546916961669922, 'token': 3995, 'token_str': 'doctor', 'sequence': 'In the hospital the man works as a doctor.'}, {'score': 0.06976664066314697, 'token': 23722, 'token_str': 'cleaner', 'sequence': 

In [ ]:
print(fill_mask("The only destiny of a woman is to be a [MASK]."))
print(fill_mask("The only destiny of a man is to be a [MASK]."))

[{'score': 0.07571644335985184, 'token': 1590, 'token_str': 'woman', 'sequence': 'The only destiny of a woman is to be a woman.'}, {'score': 0.052465204149484634, 'token': 6748, 'token_str': 'slave', 'sequence': 'The only destiny of a woman is to be a slave.'}, {'score': 0.03437161073088646, 'token': 21803, 'token_str': 'prostitute', 'sequence': 'The only destiny of a woman is to be a prostitute.'}, {'score': 0.02620248682796955, 'token': 8229, 'token_str': 'warrior', 'sequence': 'The only destiny of a woman is to be a warrior.'}, {'score': 0.023374376818537712, 'token': 7559, 'token_str': 'lover', 'sequence': 'The only destiny of a woman is to be a lover.'}]
[{'score': 0.043410658836364746, 'token': 6748, 'token_str': 'slave', 'sequence': 'The only destiny of a man is to be a slave.'}, {'score': 0.04164069890975952, 'token': 5540, 'token_str': 'god', 'sequence': 'The only destiny of a man is to be a god.'}, {'score': 0.035219352692365646, 'token': 1590, 'token_str': 'woman', 'sequence

In [ ]:
# try something
get_fillers("")

Telicity: a simple example of a linguistic property

In [ ]:
s1 = "I walked [MASK] 3 hours."
s2 = "I finished [MASK] 3 hours."
get_fillers(s1)
get_fillers(s2)

I walked for 3 hours.
I finished in 3 hours.


What about ambigous contexts?

In [ ]:
# context is important!
get_fillers("I ate everything [MASK] 3 hours")
get_fillers("I ate everything [MASK] just 3 hours")


I ate everything for 3 hours


Does it respect pronoun agreement?

In [ ]:
# pronoun gender
get_fillers("My teacher name is Daniel. [MASK] is a very good teacher.")

My teacher name is Daniel. he is a very good teacher.


In [ ]:
get_fillers("John and Mary went out of the cinema and suddenly he kissed [MASK].")

John and Kevin went out of the cinema and suddenly he kissed her.


In [ ]:
get_fillers("Lucy and mary went out of the cinema and suddenly she kissed [MASK].")

Lucy and mary went out of the cinema and suddenly she kissed him.


Subject-Verb agreement

In [ ]:
get_fillers("The boys [MASK] shy.", output_all = True)

[{'score': 0.41352078318595886,
  'token': 1132,
  'token_str': 'are',
  'sequence': 'The boys are shy.'},
 {'score': 0.25092068314552307,
  'token': 1127,
  'token_str': 'were',
  'sequence': 'The boys were shy.'},
 {'score': 0.06096593663096428,
  'token': 3166,
  'token_str': 'seem',
  'sequence': 'The boys seem shy.'},
 {'score': 0.026996642351150513,
  'token': 3118,
  'token_str': 'remain',
  'sequence': 'The boys remain shy.'},
 {'score': 0.01755758561193943,
  'token': 2845,
  'token_str': 'appear',
  'sequence': 'The boys appear shy.'}]

## Question Answering

To solve Q&A tasks we must call a fine-tuned model. Here the *squad* in the model id stands for *Stanford Questions and Answering Dataset*. Which means the model has been fine-tuned on that dataset



In [ ]:
#instantiate the qa pipeline
qa = pipeline("question-answering", "distilbert/distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
#helper fucntion to print only the answer
def get_answer(query, context):
  return qa(query, context)["answer"]

In [ ]:
question =  "Where did I leave the keys?"
context = "I left the keys on the table before leaving home yesterday"
qa(question, context)

{'score': 0.307202011346817, 'start': 16, 'end': 28, 'answer': 'on the table'}

In [ ]:
question =   "When I came home yesterday?"
context = "Yesterday I went home late at night, after a drink with my friends"

get_answer(question, context)

'late at night'

"Exercise"

You can try to fill the quotes below with some text of your choice.

In [ ]:
question =   ""
context = ""
get_answer(question, context)

## Classification
*Adapted from https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/*


![img](https://miro.medium.com/v2/resize:fit:720/format:webp/1*fOdb7SbBeMyWcHfTuVyeQA.png)

*Picture from https://towardsdatascience.com/feature-extraction-with-bert-for-text-classification-533dde44dc2f*

We can use BERT as a feature extractor. This means to use it to generate vector representation of our texts. We can then feed a *ML* algorithm with our representations and train it to solve a *classification task*

Whe we extract features from a list of sentences we have to do several things:
- Tokenize each sentence
- Truncate sentences longer than the available context (512 tokens for bert)
- Pad the seqences with zeros to make every sentence long as the longest

For a detailed exposition of these steps we refer to [Jay Alammar's blog](https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/) and the notebook provided there


Here, for the sake of simplicity we compress all the steps (and even jump some)

In [ ]:
#instantiate the feature extraction pipeline
feature_extractor = pipeline("feature-extraction", model_id)


We download a dataset containing movie reviews tagged with a value which could be 0 or 1.

- 0 stands for Negative
- 1 stands for positive

*Remember that every ML model deals only with numbers! It is our task to map words or textual label into numbers and viceversa*

In [ ]:
# download an example dataset
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)[:2000]
df.columns = ["text","label"]
df.head()

,text,label
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


The steps we are going to follow are:
- Feature extraction. We extract the embeddings for each of our texts
    
  - after that we are going to extract and store only the embedding corresponding to the [CLS] token. This will act as a global representation of our whole text.
- We divide our extracted features in two separate sets:

 - training_set = this is used to train our models along with the training labels
 - test_set = we will evaluate our trained model on this small test, using the test_labels
- we feed a Machine Learning classifier with our features and the corresponding label. This ML model will learn to categorize the representations we provide to it assigning a value of 0 or 1. 0 stands for *Negative*, 1 stands for *Positive*
-We evaluate the *Accuracy* of our model trained on the extracted representation and compare it to a dummy baseline.





<div>
<img src = https://jalammar.github.io/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png width = 800>
</div>

<div>
<img src = https://jalammar.github.io/images/distilBERT/bert-distilbert-train-test-split-sentence-embedding.png width = 800>
</div>


<div>
<img src = https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png width = 800 >
</div>

*Pictures from https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/*








In [ ]:
#esxtract features with our feature extractor
features = feature_extractor(df["text"].tolist(), return_tensors = "pt")

In [ ]:
# extract the classification token [CLS] of each sentence
cls_tokens = np.array([f.squeeze(dim = 0)[0].numpy() for f in features])

In [ ]:
# split the data in train and test set
labels = df["label"]
train_features, test_features, train_labels, test_labels = train_test_split(cls_tokens, labels)

In [ ]:
#train a logistic classifier with distil-bert representations
log_reg = LogisticRegression()
log_reg.fit(train_features, train_labels)

#evaluate our model on test set computing the accuracy
print(f"Distil BERT score: {log_reg.score(test_features, test_labels)}")

Distil BERT score: 0.82


In [ ]:
# compare results with a dummy classifier that goes randomly
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()
scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f " % (scores.mean()))

Dummy classifier score: 0.525 


### We can then use the classifier trained on  BERT representations to predict new text.

*Note* that the text has to undergo the same process as the training phase!
that is it should be transformed into a vector representation in the first place.

I have provided a simple function to do so


In [ ]:
# helper function to get pretty printed predictions predictions
def predict(text: str):
  cls_token = feature_extractor(text)[0]
  pred = log_reg.predict(cls_token)
  if pred[0] == 0:
    print("NEGATIVE")
  else:
    print("POSITIVE")


predict("i hate the film")
predict("I loved the film")


NEGATIVE
POSITIVE


In [ ]:
predict("This film was a total crap")


NEGATIVE


Let's try it on real [reviews](https://www.rottentomatoes.com/m/poor_things/reviews?type=user) taken on rotten tomatoes about the movie *Poor Things*


ex1: 1/5 *

best to skip this weird piece of blank. apart from one 90 second scene with Dafoe and Stone when she returns to the mansion to learn about "God's" reasons, the movie is completely mishandled and skippable.

---
ex2: 5/5 *

One of the most glorious examples of surrealist cinema...really my pride in the fact that a Greek director is behind such a brilliant idea makes me very proud.

---

ex3: 3,5/5 *

The production design team deserve heaps of praise, absolutely stunning sets. The overall story felt like it lacked depth however and I personally took nothing from the film and found it rather forgettable. Visually quite stunning at times although occasionally a little gimmicky.


In [ ]:
#store the reviews
review1 = "best to skip this weird piece of blank. apart from one 90 second scene with Dafoe and Stone when she returns to the mansion to learn about God's reasons, the movie is completely mishandled and skippable."
review2 = "One of the most glorious examples of surrealist cinema...really my pride in the fact that a Greek director is behind such a brilliant idea makes me very proud."
review3 = "The production design team deserve heaps of praise, absolutely stunning sets. The overall story felt like it lacked depth however and I personally took nothing from the film and found it rather forgettable. Visually quite stunning at times although occasionally a little gimmicky."

In [ ]:
#predict the sentiment for review1
predict(review1)

NEGATIVE


In [ ]:
#predict review 2
predict(review2)

POSITIVE


In [ ]:
#predict review3
predict(review3)

POSITIVE


---
# GPT

<div>
  <img src="https://lena-voita.github.io/resources/lectures/lang_models/neural/nn_lm_idea_linear-min.png" width="800">
</div>


Causal language modeling is done by feeding the moel with the seqeunce and the real expected continuation as labels. This is reapeted recursively for each step of the sequence, as shown in the image below:

![img](https://jalammar.github.io/images/gpt2/transformer-decoder-attention-mask-dataset.png)

In [ ]:
sent = "My name is Bond, James"

In [ ]:
# download the model
gpt2_model_id = "gpt2"
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_id)
gpt_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_id)

In [ ]:
#encode the text for GPT
gpt_encs = gpt_tokenizer(sent, return_tensors = "pt", add_special_tokens = True)

In [ ]:
# take a look at the tokenization output
for n, id in enumerate(gpt_encs["input_ids"][0]):
  print(n,tokenizer.decode(id),"---->", id.item())

In [ ]:
#generate the output
output = gpt2_model.generate(**gpt_encs, max_new_tokens = 1)
output
tokenizer.batch_decode(output)

We could try to train a logistic classifier in the same way we did with BERT with the GPT2 representations

### Text generation pipeline

In [ ]:
#instantiate the text generation pipeline
text_gen = pipeline('text-generation', model='gpt2', pad_token_id = 50256)

In [ ]:
# exmple of raw generation for 5 sequences
text_gen("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

What can we control about the generation?

We can set a few parameters to give a minumum direction to our generation.
- temperature = a paramenter taking float values between 0 and 1. It regulates the creativity of the model generations. A value closer to 1, make the model more creative but less reliable. On the opposite, going toward 0, makes the model more deterministic, but more accurate.
- max_new_tokens = the number of new tokens we want to generate
- num_return_sequences = the number of sequences we want to return.

More parameters are available such as *top_k* and *top_p*, but they concern more advanced generation decoding settings.

In [ ]:
#helper function to get results pretty printed
def get_gens(text, output_all = False, temperature = 0.1, max_new_tokens = 1, num_return_sequences = 1):
  if not output_all:
    print(text_gen(text,
                   max_new_tokens = max_new_tokens,
                   temperature = temperature,
                   num_return_sequences = num_return_sequences
                   )[0]["generated_text"])
  else:
    return text_gen(text, max_new_tokens = max_new_tokens, temperature = temperature, pad_token_id = 50256 )

Indirect oject identification

In [ ]:
get_gens("John and Mary went out of the cinema when suddenly he kissed")

In [ ]:
get_gens("John and Mary went out of the cinema when suddenly she kissed")

Subject-Verb agreement

In [ ]:
get_gens("I saw the boys. He")

In [ ]:
get_gens("I saw the boys. They")